In [ ]:
# Standard libraries
import numpy as np
import h5py
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interactive

# Append base directory
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
path1p = os.path.dirname(currentdir)
path2p = os.path.dirname(path1p)
libpath = os.path.join(path2p, "lib")

sys.path.insert(0, libpath) 
print("Appended library directory", libpath)

# User libraries
from signal_lib import resample
from data_io.qt_wrapper import gui_fnames, gui_fpath
from data_io.yaro.yaro_te_h5_lib import readTE_H5, getStatistics, parseTEfolders, flatten_param_sweep
import plots.te_plots as te_plots

%load_ext autoreload
%autoreload 2

## Processing single point data

In [ ]:
# Get datasets and parse them
path_tmp = "/mnt/hifo_scratch1/Yaro/TE_from_Aleksejs/data_idtxl/swipe/specific-param-study"
parsed_te_datasets_dict = parseTEfolders(path_tmp)

In [ ]:
# Read data
for dataname, dataval in parsed_te_datasets_dict.items():
    dataval['data'] = np.array([readTE_H5(filepath) for filepath in dataval["filepaths"]])

## Swipe Analysis

**TODO**:
1) Plot: Diff nlinks between two datasets
2) Plot: Shared/Overlap links between two datasets

**Later**
1) Convert storage to PANDAS
2) Adequately rename folders in structure
3) Separate plots from main code, save figures to file.

In [ ]:
def timestep_from_filename(fname):
    if "raw" in fname:
        return 0.05 # seconds
    elif "subsample" in fname:
        return 0.2 # seconds
    else:
        raise ValueError("Can't figure out timestep from", fname)

In [ ]:
# Get output folder
outpath = gui_fpath("Select folder for output images", "./")

In [ ]:
#################################
# time : Total Number of links - By Timestep
#################################
for dataname, dataval in parsed_te_datasets_dict.items():
    timestep = timestep_from_filename(dataname)
    
    # Precompute parameter sweep
    param_sweep_keys = ['isAnalysis', 'isMouse', 'isTrial', 'isRange', 'isMethod']
    param_sweep_dicts = [dataval["statistics"][key] for key in param_sweep_keys]
    param_sweep = flatten_param_sweep(param_sweep_dicts, dataval["summary"]["dataname"])

    for title_suffix, idxs in zip(*param_sweep):
        data_lst  = dataval['data'][idxs.astype(bool)]
        label_lst = dataval['filenames'][idxs.astype(bool)]
        
        # Plotting connection number
        outfname = os.path.join(outpath, "nconn_bytime_" + title_suffix + ".png")
        te_plots.plot_te_nconn_bytime(outfname, data_lst, label_lst, timestep)

In [ ]:
################################################
# Session : Total number of links - Range By Days
################################################
time2step = lambda t:  int(t / 10.0 * N_TIMES)   # FIXME

ranges_sec  = {'TEX' : (2.0, 3.5), 'LIK' : (3.5, 6.0)}
ranges_step = {k, (times2step(v[0]), times2step(v[1])) for k,v in ranges_sec.items()}

for dataname, dataval in parsed_te_datasets_dict.items():
    timestep = timestep_from_filename(dataname)
    
    # Precompute parameter sweep
    param_sweep_keys = ['isAnalysis', 'isMouse', 'isTrial', 'isMethod']
    param_sweep_dicts = [dataval["statistics"][key] for key in param_sweep_keys]
    param_sweep = flatten_param_sweep(param_sweep_dicts, dataval["summary"]["dataname"])

    for title_suffix, idxs in zip(*param_sweep):
        data_lst  = dataval['data'][idxs.astype(bool)]
        label_lst = dataval['filenames'][idxs.astype(bool)]
        
        # Plotting connection number
        for rng_name, rng_sec in ranges_sec.items():
            outfname = os.path.join(outpath, "nconn_rangebytime_" + rng_name + "_" + title_suffix + ".png")
            te_plots.plot_te_nconn_rangebydays(outfname, data_lst, label_lst, ranges_step, timestep)

In [ ]:
#################################
# Session : Shared Link Scatter
#################################
for dataname, dataval in parsed_te_datasets_dict.items():
    timestep = timestep_from_filename(dataname)
    
    # Precompute parameter sweep
    param_sweep_keys = ['isAnalysis', 'isMouse', 'isTrial', 'isMethod']
    param_sweep_dicts = [dataval["statistics"][key] for key in param_sweep_keys]
    param_sweep = flatten_param_sweep(param_sweep_dicts, dataval["summary"]["dataname"])

    for title_suffix, idxs in zip(*param_sweep):
        data_lst  = dataval['data'][idxs.astype(bool)]
        label_lst = dataval['filenames'][idxs.astype(bool)]
        
        # Plotting connection number
        outfname = os.path.join(outpath, "shared_link_scatter_" + title_suffix + ".png")
        te_plots.plot_te_shared_link_scatter(outfname, data_lst, label_lst, timestep)

In [ ]:
#################################
# Distribution of TE
#################################
timestep = 0.2 # FIXME

for dataname, dataval in parsed_te_datasets_dict.items():
    timestep = timestep_from_filename(dataname)
    
    # Precompute parameter sweep
    param_sweep_keys = ['isAnalysis', 'isMouse', 'isTrial', 'isRange', 'isMethod']
    param_sweep_dicts = [dataval["statistics"][key] for key in param_sweep_keys]
    param_sweep = flatten_param_sweep(param_sweep_dicts, dataval["summary"]["dataname"])

    for title_suffix, idxs in zip(*param_sweep):
        data_lst  = dataval['data'][idxs.astype(bool)]
        label_lst = dataval['filenames'][idxs.astype(bool)]
        
        # Plotting TE distribution
        outfname = os.path.join(outpath, "te_distr_" + title_suffix + ".png")
        te_plots.plot_te_distribution(outfname, data_lst, label_lst, timestep)

In [ ]:
#################################
# time : Relative difference between number of links
#################################
def comp_rel_diff_nconn(dataname1, dataname2):
    dt1 = timestep_from_filename(dataname1)
    dt2 = timestep_from_filename(dataname2)
    
    dataval1 = parsed_te_datasets_dict[dataname1]
    dataval2 = parsed_te_datasets_dict[dataname2]
    dataname_diff = dataname1 + "_" + dataname2
    
    # Precompute parameter sweep
    param_sweep_keys = ['isAnalysis', 'isMouse', 'isTrial', 'isRange', 'isMethod']
    param_sweep_dicts1 = [dataval1["statistics"][key] for key in param_sweep_keys]
    param_sweep_dicts2 = [dataval1["statistics"][key] for key in param_sweep_keys]
    param_sweep1 = flatten_param_sweep(param_sweep_dicts, dataname_diff)
    param_sweep2 = flatten_param_sweep(param_sweep_dicts, dataname_diff)
    assert len(param_sweep1)==len(param_sweep2), "All names in two datasets must match"
    
    for title_suffix, idxs in zip(*param_sweep1):
        data_lst1  = dataval1['data'][idxs.astype(bool)]
        data_lst2  = dataval2['data'][idxs.astype(bool)]
        label_lst1 = dataval1['filenames'][idxs.astype(bool)]
        label_lst2 = dataval2['filenames'][idxs.astype(bool)]
        
        # Plotting TE distribution
        outfname = os.path.join(outpath, "nconn_diff_bytime_" + title_suffix + ".png")
        te_plots.plot_te_rel_diff_nlinks_bytime(outfname, data_lst1, data_lst2, label_lst1, label_lst2, dt1, dt2)
        
comp_rel_diff_nconn('raw_delay_2_window_3',       'raw_delay_2_window_10')
comp_rel_diff_nconn('raw_delay_2_window_3',       'subsample_delay_2_window_6')
comp_rel_diff_nconn('raw_delay_2_window_10',      'subsample_delay_5_window_6')
comp_rel_diff_nconn('subsample_delay_2_window_6', 'subsample_delay_5_window_6')

In [ ]:
#################################
# Degree Distribution
#################################


In [ ]:
'''
  For each dataset, for each file, for each param combination
'''
import pandas as pd

# Neuron Labels
#labels = ['Cpu', 'GP', 'Rt', 'S1_bf', 'LD', 'VM', 'VL', 'CA1', 'CA1_Py', 'CA1_Mol', 'DG', 'M1']
labels = ['Cpu','GP','Po','S1_bf','VM','VL','LDVL','M1','CA1_Py','LHb','cc','cc']
pd.DataFrame({i : l for i,l in enumerate(labels)}, index=["Region Name"])

# # Behaviour Labels
# labels = ['lick', 'paw', 'Start Cue', 'End Cue', 'Reward', 'First Touch']
# pd.DataFrame({i : l for i,l in enumerate(labels)}, index=["Behaviour"])

## Range Analysis

In [ ]:
isCorrectMouse = np.array([mname == 'mtp_15' for mname in mouse_names], dtype=int)

fig1, ax1 = plt.subplots(ncols=2, figsize=(10, 5))
fig2, ax2 = plt.subplots(nrows=2, ncols=3, figsize=(15, 5), tight_layout=True)

for i, trial in enumerate(["ALL"]):
    for j, method in enumerate(["BTE","MTE"]):
        ax2[j][0].set_ylabel(method)
        
        for k, rng in enumerate(["CUE", "TEX", "LIK"]):
            ax2[0][k].set_title(rng)
            
            idxs_ths = isCorrectMouse + isAnalysis["range"] + isTrial[trial] + isMethod[method] + isRange[rng] == 5
            print("For trials", trial,"range", rng,"method", method, "have", np.sum(idxs_ths), "files")
            
            nConn = []
            actSum = np.zeros((12,12))
            for fname, basename in zip(datafilenames[idxs_ths], basenames[idxs_ths]):
#                 print("Processing file: ", basename)
                te, lag, p = getData(fname)

                isActive = 1-np.isnan(te).astype(int)
                actSum += isActive
                nConn += [np.sum(isActive) / (te.shape[0]**2 - te.shape[0])]
#                 totalConnPerConn[-1][-1].append(np.sum(1-np.isnan(te).astype(int), axis=2).flatten()  / (te.shape[0]**2 - te.shape[0]))

            ax1[j].plot(nConn, label=rng)
            ax2[j][k].imshow(actSum)
    
        ax1[j].set_title(method)
        ax1[j].set_xlabel("day/file")
        ax1[j].set_ylabel("ratio of active connections")
        ax1[j].legend()        

In [ ]:
# for model in ["BivariateTE", "MultivariateTE"]:
#     for i in range(16, 20):
#         for rng in ["CUE", "TEX", "LIK"]:
#             fname = "mtp_15_2018_05_" + str(i) + "_a_" + model + "_range_" + rng + ".h5"
#             te, lag, p = getData(fname)
            
#             fig, ax = plt.subplots(ncols = 3, figsize=(15, 5))
#             ax[0].imshow(te[:,:], cmap="jet", vmin=0, vmax=1)
#             ax[1].imshow(lag[:,:], cmap="jet", vmin=1, vmax=5)
#             ax[2].imshow(p[:,:], cmap="jet", vmin=0, vmax=1)
#             ax[0].set_title("TE")
#             ax[1].set_title("delay")
#             ax[2].set_title("p-value")
            
#             fig.suptitle(fname)
#             plt.show()

## Processing swipe Data

In [ ]:
bte_te, bte_lag, bte_p = getData("mtp_15_2018_05_19_a_BivariateTE_swipe.h5")
mte_te, mte_lag, mte_p = getData("mtp_15_2018_05_19_a_MultivariateTE_swipe.h5")

# Interactive
def f(i):
    fig, ax = plt.subplots(nrows = 2, ncols = 3, figsize=(12, 8))
    ax[0][0].imshow(bte_te[:,:,i], cmap="jet", vmin=0, vmax=1)
    ax[1][0].imshow(mte_te[:,:,i], cmap="jet", vmin=0, vmax=1)
    ax[0][1].imshow(bte_lag[:,:,i], cmap="jet", vmin=1, vmax=5)
    ax[1][1].imshow(mte_lag[:,:,i], cmap="jet", vmin=1, vmax=5)
    ax[0][2].imshow(bte_p[:,:,i], cmap="jet", vmin=0, vmax=1)
    ax[1][2].imshow(mte_p[:,:,i], cmap="jet", vmin=0, vmax=1)
    
    ax[0][0].set_ylabel("Bivariate")
    ax[1][0].set_ylabel("Multivatiate")
    ax[0][0].set_title("TE")
    ax[0][1].set_title("delay")
    ax[0][2].set_title("p-value")
    plt.show()
    
interactive_plot = interactive(f, i=(0, bte_te.shape[2]-1, 1))
output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot